## **Hyperparameter Tuning: Neighborhood-Based Collaborative Filtering**

### **Import necessary library**

In [11]:
import pandas as pd 
import numpy as np 
import math 
from collaborative_filtering import build_utility_matrix, neighborhood_hyperparameter_tuning

In [3]:
import pandas as pd

part_ids = range(2, 11)

# Read the first file
data = pd.read_csv('resources/data/split_ratings/ratings_part_1.txt', header=None)

# Loop through other files
for id in part_ids:
    file_path = f'resources/data/split_ratings/ratings_part_{id}.txt'
    try:
        tmp = pd.read_csv(file_path, on_bad_lines="skip", header=None)  # Skip problematic rows
        data = pd.concat([data, tmp], ignore_index=True)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

data.columns =['UserID', 'MovieID', 'Rating']

# data = pd.read_csv('resources/data/split_ratings/ratings_part_1.txt', header=None)
# data.columns =['UserID', 'MovieID', 'Rating']

In [4]:
data = data.sort_values(by=['UserID', 'MovieID'])
data = data.reset_index()
data = data.drop(columns=['index'])

In [5]:
data.head()

,UserID,MovieID,Rating
0,66,68646,10
1,66,71562,10
2,66,76759,10
3,66,80684,10
4,66,88763,8


In [6]:
data.tail()

,UserID,MovieID,Rating
4680501,190007800,8115900,7
4680502,190007800,10872600,10
4680503,190007800,12412888,9
4680504,190007800,19623240,1
4680505,190007800,21235248,8


In [7]:
# # Count the number of interactions for each movie
# movie_counts =data['MovieID'].value_counts()

# # Filter out movies appear < 10 times 
# movies_to_keep = movie_counts[movie_counts >= 10].index
# data = data[data['MovieID'].isin(movies_to_keep)]

# # Count the number of interactions for each user
# user_counts = data['UserID'].value_counts()

# # Filter out users appear < 10 times 
# users_to_keep = user_counts[user_counts >= 10].index 
# data = data[data['UserID'].isin(users_to_keep)]

In [8]:
num_users = data.UserID.unique().shape[0]
num_movies = data.MovieID.unique().shape[0]
num_users, num_movies

(11381, 9814)

### **Split data**

In [9]:
def split_data(df, test_size=0.3, random_state=42):
    np.random.seed(random_state)
    
    # Preprocessing Step: Remove movies with frequency = 1
    movie_counts = df['MovieID'].value_counts()
    movies_to_keep = movie_counts[movie_counts > 1].index
    df = df[df['MovieID'].isin(movies_to_keep)].reset_index(drop=True)
    
    # Initialize sets to keep track of test indices
    test_indices = set()
    
    # Ensure each user has at least one entry in test set
    for user_id, group in df.groupby('UserID'):
        if len(group) >= 1:
            test_count = max(1, int(len(group) * test_size))
            test_sample = np.random.choice(group.index, size=test_count, replace=False)
            test_indices.update(test_sample)
    
    # Ensure each movie has at least one entry in test set
    for movie_id, group in df.groupby('MovieID'):
        # Find indices not already in test set
        remaining_indices = list(set(group.index) - test_indices)
        if remaining_indices:
            test_sample = np.random.choice(remaining_indices, size=1, replace=False)
            test_indices.update(test_sample)
    
    # Create test and train datasets
    test_df = df.loc[list(test_indices)].reset_index(drop=True)
    train_df = df.drop(list(test_indices)).reset_index(drop=True)
    
    # Optional: Verify that both train and test have the same users and movies
    common_users = set(train_df['UserID']).intersection(set(test_df['UserID']))
    common_movies = set(train_df['MovieID']).intersection(set(test_df['MovieID']))
    
    train_df = train_df[train_df['UserID'].isin(common_users) & train_df['MovieID'].isin(common_movies)].reset_index(drop=True)
    test_df = test_df[test_df['UserID'].isin(common_users) & test_df['MovieID'].isin(common_movies)].reset_index(drop=True)
    
    return train_df, test_df

# Example usage:
# Assuming your dataframe is named `df` and has columns 'UserID' and 'MovieID'
# train, test = split_data(df, test_size=0.2, random_state=42)


In [10]:
train_data, test_data = split_data(data)

KeyboardInterrupt: 

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
num_users = train_data.UserID.unique().shape[0]
num_movies = train_data.MovieID.unique().shape[0]
num_users, num_movies

In [ ]:
test_data.head()

In [ ]:
test_data.tail()

In [ ]:
num_users = test_data.UserID.unique().shape[0]
num_movies = test_data.MovieID.unique().shape[0]
num_users, num_movies

In [33]:
movies = train_data.MovieID.drop_duplicates()
movies = pd.DataFrame(movies, columns=['MovieID'])
movies = movies.sort_values('MovieID', ascending=True)
movies = movies[['MovieID']]
movies = movies.reset_index()

In [34]:
users = train_data.UserID.drop_duplicates()
users = pd.DataFrame(users, columns=['UserID'])
users = users.sort_values('UserID', ascending=True)
users = users[['UserID']]
users = users.reset_index()

### **Build Utility Matrix**

In [ ]:
utility_matrix = build_utility_matrix(train_data)
utility_matrix

### **Defining the Hyperparameter Grid**

In [19]:
from itertools import product

# Define the hyperparameter grid
k_neighbors = [10, 15, 20, 25, 30, 35, 40, 45, 50]
uu_cf = [True, False]
cosine = [True, False]

# Create all possible combinations of hyperparameters
hyperparameter_combinations = list(product(k_neighbors, uu_cf, cosine))


### **Evaluating each combination**

In [ ]:
results_df = neighborhood_hyperparameter_tuning(
    R=utility_matrix,
    hyperparameter_combinations=hyperparameter_combinations,
    validation_ratio=0.2
)

In [ ]:
# Find the combination with the lowest validation RMSE
best_result = results_df.loc[results_df['validation_RMSE'].idxmin()]
print("Best Hyperparameters:")
print(best_result)